In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import re
import requests
import math
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg
import matplotlib.figure
import seaborn as sb
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
'''listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=2').content, 'lxml')
listings_soup.find_all('h4', {'class': 'heading-3 margin-y-1 font-weight-bold'})'''
page1_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/').content, 'lxml')
page1_prices_soup = page1_listings_soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
page1_prices_soup

[<h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$13,990</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$18,870</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$18,093</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$26,500</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$7,995</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$26,419</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$30,475</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$7,995</h4>,
 <h4 class

In [3]:
pricespage1 = re.findall('[0-9]+,[0-9]+', str(page1_prices_soup))
pricespage1

['13,990',
 '18,870',
 '18,093',
 '26,500',
 '7,995',
 '26,419',
 '30,475',
 '7,995',
 '9,991',
 '9,888',
 '28,300',
 '8,799',
 '7,991',
 '29,940',
 '31,945',
 '12,924',
 '30,680',
 '32,977',
 '30,998',
 '32,011',
 '21,997',
 '13,000',
 '27,550',
 '26,988',
 '24,995',
 '29,999',
 '9,991',
 '15,995',
 '27,950',
 '9,000',
 '8,000',
 '25,488',
 '27,888']

In [4]:
page1_prices = []
for price in pricespage1:   
    page1_prices.append(str(price).replace(',', '.'))
page1_prices

['13.990',
 '18.870',
 '18.093',
 '26.500',
 '7.995',
 '26.419',
 '30.475',
 '7.995',
 '9.991',
 '9.888',
 '28.300',
 '8.799',
 '7.991',
 '29.940',
 '31.945',
 '12.924',
 '30.680',
 '32.977',
 '30.998',
 '32.011',
 '21.997',
 '13.000',
 '27.550',
 '26.988',
 '24.995',
 '29.999',
 '9.991',
 '15.995',
 '27.950',
 '9.000',
 '8.000',
 '25.488',
 '27.888']

In [ ]:
#Scraping rest of pages listings prices

other_pages_prices = []
for number in range(2, 241):
    page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number)).content, 'lxml')
    page_prices_soup = page_listings_soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
    page_prices = list(re.findall('[0-9]+,[0-9]+', str(page_prices_soup)))
    other_pages_prices += page_prices
other_pages_prices

In [ ]:
rest_pages_prices = []
for price in other_pages_prices:   
    rest_pages_prices.append(price.replace(',', '.'))
rest_pages_prices

In [ ]:
prices = page1_prices + rest_pages_prices
len(prices)